In [ ]:
from IPython.display import display, clear_output


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import P7_DataBreed
import p5_util

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from  sklearn import model_selection
import numpy as np

#-------------------------------------------------------------------------------
#
#-------------------------------------------------------------------------------
def arr_X_y_build(X,y):
    list_X = [np.array(x) for x in X]
    list_y = [Y for Y in y]

    weight  = list_X[0].shape[0]
    height  = list_X[0].shape[1]
    channel = list_X[0].shape[2]
    
    arr_X  = list_X[0].reshape((1, weight, height, channel))
    

    image_count = X.shape[0]
    list_image_error = list()
    for image_i in range(1,image_count) :
        try :
            arr_X\
            = np.append(arr_X,list_X[image_i].reshape((1, weight, height, channel)),axis=0)
        except ValueError :
            list_image_error.append(image_i)

    arr_y = np.array(y)
    return arr_X, arr_y
#-------------------------------------------------------------------------------

#-------------------------------------------------------------------------------
#
#-------------------------------------------------------------------------------
def arr_X_train_test_build(X,y,test_size=0.2):
    X_train, X_test, y_train,  y_test \
    = model_selection.train_test_split(X,y,test_size=test_size)

    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    print("Images for training...")
    arr_X_train, arr_y_train = arr_X_y_build(X_train, y_train)
    print("Images for testing...")
    arr_X_test, arr_y_test = arr_X_y_build(X_test, y_test)
    return arr_X_train, arr_X_test, arr_y_train, arr_y_test
#-------------------------------------------------------------------------------

In [ ]:
import p5_util
filename= './data/df_breed_image.dump'
df = p5_util.object_load(filename)
df.shape

**Execute following cell in order to update object version with no change in attributes**

* A backup instance of object is created. This backup instance is empty.
* If object instance is not defined then it is read from dmped file.
* object instance is copied back into backup instance
* Backup instance is deleted

In [ ]:
new_attribute=None

import p5_util
import P7_DataBreed

if new_attribute is None :
    is_saved = True
    filename='./data/oP7_DataBreed.dump'
    oP7_DataBreed_save = P7_DataBreed.P7_DataBreed()
    try: 
        oP7_DataBreed
    except NameError:
        print('*** INFO : oP7_DataBreed is not defined; loading...')
        oP7_DataBreed = p5_util.object_load(filename)
        is_saved = False
    oP7_DataBreed_save.copy(oP7_DataBreed)
    oP7_DataBreed = P7_DataBreed.P7_DataBreed()
    oP7_DataBreed.copy(oP7_DataBreed_save)
    del(oP7_DataBreed_save)
    oP7_DataBreed.show()
    is_saved=True
    if is_saved is True:
        print('*** INFO : oP7_DataBreed is saved')
        p5_util.object_dump(oP7_DataBreed,filename)


**Building Data-model**

DataFrame with columns as images, breed names and labels is built.

Labels are issued from encoding breed names. 

Labels are then used for training Keras network. 


In [ ]:
df = oP7_DataBreed.df_build()

**Dataframe is saved**

In [ ]:
import p5_util

filename= './data/df_breed_image.dump'
p5_util.object_dump(df,filename)
df.sample()

In [ ]:
df.shape, df.columns

### Image truncated against image resize

In [ ]:
ser_pil_image = df['image']
ser_label = df['label']

#### Original image sample

In [ ]:
pil_image = ser_pil_image.iloc[0]
pil_image

#### Resized image

In [ ]:
from PIL import Image
pil_image.resize((204,204))

#### Truncated image as square

In [ ]:
import numpy as np
import P7_DataBreed
square=(204,204)
pil_image_truncated = P7_DataBreed.pil_square(ser_pil_image.iloc[0], square=square)
print(np.array(pil_image_truncated).shape)
#arr_keras_X = arr_keras_X.reshape(1,arr_keras_X.shape[0],arr_keras_X.shape[1],arr_keras_X.shape[2])
pil_image_truncated

In [ ]:
pil_image_truncated

### Building train and test sample, Keras compliant

In [ ]:
import P7_DataBreed

arr_keras_image_train, arr_keras_image_test, arr_label_train, arr_label_test \
= P7_DataBreed.p7_keras_X_train_test_build(ser_pil_image, ser_label, test_size=0.2, square=(204,204))

arr_keras_image_train.shape, arr_keras_image_test.shape, arr_label_train.shape, arr_label_test.shape

#### Dump train and test sample

In [ ]:
import p5_util
filename='./data/arr_keras_X_y_train_test.dump'
p5_util.object_dump((arr_keras_image_train, arr_keras_image_test, arr_label_train, arr_label_test),filename)

### Building Keras network

#### Read dumped file for train and test sample

In [ ]:
import p5_util
filename='./data/arr_keras_X_y_train_test.dump'
(arr_X_train,arr_X_test, arr_y_train, arr_y_test) = p5_util.object_load(filename)
print(arr_X_train.shape,arr_X_test.shape,arr_y_train.shape,arr_y_test.shape)

#### Building Keras layers

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense

my_VGG16 = Sequential()  # Création d'un réseau de neurones vide 

# Ajout de la première couche de convolution, suivie d'une couche ReLU
w=arr_X_train.shape[1]
h=arr_X_train.shape[2]
c=arr_X_train.shape[3]
print(w,h,c)
my_VGG16.add(Conv2D(64, (3, 3), input_shape=(w, h, c), padding='same', activation='relu'))

In [ ]:
# Ajout de la deuxième couche de convolution, suivie  d'une couche ReLU
my_VGG16.add(Conv2D(64, (3, 3), padding='same', activation='relu'))

In [ ]:
# Ajout de la première couche de pooling
my_VGG16.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))


In [ ]:
import numpy as np

# Ajout de la dernière couche fully-connected qui permet de classifier
my_VGG16.add(Flatten())  # Conversion des matrices 3D en vecteur 1D
nb_labels = arr_y_train.shape[0]
print("Number of labels for training process : "+str(nb_labels))
my_VGG16.add(Dense(nb_labels, activation='softmax'))


In [ ]:
from keras import optimizers
sgd = optimizers.SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)

#my_VGG16.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
my_VGG16.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [ ]:
#for X_train, y_train in zip(list_X_train, list_y_train):
print(arr_X_train.shape)
my_VGG16.fit(arr_X_train, arr_y_train)

**Test for images classification**

In [ ]:
arr_result = my_VGG16.predict(arr_X_test)

In [ ]:
filename= './data/result_vgg16.dump'
p5_util.object_dump(arr_result,filename)

In [ ]:
arr_result.shape,arr_y_train.shape

In [ ]:
np.unique(arr_y_train).shape[0]

In [ ]:
np.where(arr_result>0.3)[0].shape,np.where(arr_result>0.3)[1].shape

In [ ]:
arr_result = my_VGG16.predict_classes(arr_X_test)

In [ ]:
arr_result